# Pré-processamento dos Dados
## | Avaliando Riscos do Modelo e Selecionando Variáveis Explicativas

### > **Objetivo do Notebook**: 

Este notebook é a 2ª parte do desenvolvimento do projeto focado para construção de um modelo preditivo para classificar alunos com alta chance de evasão do ensino superior. O foco desse notebook será em avaliar os riscos do modelo, dado a abordagem que será usada para a modelagem e selecionar as variáveis explicativas de maior importância através do conceito de *Feature Engineering*. 

### > **Insights da Exploração**

Com base no jupyter notebook anterior, já possuímos uma base com qual podemos começar o trabalho de avaliar as variáveis explicativas. Para avaliar essas variáveis pré-selecionadas tendo em vista o modelo de **Regressão Logística** a ser construído, vamos avaliar:

- Informação das Variáveis, calculado com Information Value (IV) baseado em Weight of Evidence (WOE) das classes para variáveis categóricas e categorizadas
- Multicolinearidade, calculado com Variance Inflation Factor (VIF)
- Outliers, calculado com Z-score
- 
Conforme o desenrolar da análise, vamos transformar as variáveis de acordo com as necessidades de obter os melhores indicadores das variáveis.

In [23]:
# Manipulação e transformação de dados
import numpy as np
import pandas as pd

# Visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

# Calculos estatísticos
from scipy.stats import pointbiserialr, f_oneway, chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

# Ferramentas de análise
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Outros
from collections import defaultdict

%matplotlib inline

In [24]:
# Listando variáveis de interesse
vars = [
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (grade)',
    'Tuition fees up to date',
    'Scholarship holder',
    'Course',
    'Application mode',
    'Target'
]

# Extraindo dados
data = pd.read_csv('../data/raw/train.csv')

# Selecionando apenas variáveis de interesse
df = data[vars].copy()

# Mapeando Target para estudar apenas o caso de evasão de alunos
df['Target'] = df['Target'].map({'Graduate': 0, 'Enrolled': 0, 'Dropout': 1})

df.head(5)

,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Tuition fees up to date,Scholarship holder,Course,Application mode,Target
0,6,14.500000,6,12.428571,1,1,9238,1,0
1,4,11.600000,0,0.000000,1,0,9238,17,1
2,0,0.000000,0,0.000000,1,0,9254,17,1
3,7,12.591250,7,12.820000,1,1,9500,1,0
4,6,12.933333,6,12.933333,1,0,9500,1,0


In [25]:
print('Numero de linhas: {0} \nNúmero de colunas: {1}'.format(df.shape[0], df.shape[1]))

Numero de linhas: 76518 
Número de colunas: 9


In [26]:
# Carregado informações sobre variáveis do notebook EDA
info = pd.read_feather('../data/processed/classificacao_variaveis_categoricas.feather')
info

,Variável,Classificação
0,Marital status,Totalmente desbalanceada
1,Application mode,Altamente desbalanceada
2,Application order,Moderadamente desbalanceada
3,Course,Totalmente desbalanceada
4,Daytime/evening attendance,Moderadamente desbalanceada
5,Previous qualification,Totalmente desbalanceada
6,Nacionality,Totalmente desbalanceada
7,Mother's qualification,Altamente desbalanceada
8,Father's qualification,Altamente desbalanceada
9,Mother's occupation,Altamente desbalanceada


O insight do notebok anterior sobre a variável **Target** tratava dela na sua forma de três classes, como nós transformarmos a variável em uma binária, abaixo vamos recalcular seu **Índice de Gini** e **Índice de Shannon** para verificar o balanceamento da variável.

In [27]:
# Função para calcular o índice de gini de acordo com a fórnula
def calcular_gini(pi):
    return 1 - np.sum(pi**2)

# Função para calcular o índice de shannon de acordo com a fórnula
def calcular_shannon(pi):
    return -np.sum(pi * np.log(pi))

In [28]:
gini = calcular_gini(df.Target.value_counts(normalize=True))
shannon = calcular_shannon(df.Target.value_counts(normalize=True))
max_shannon = np.log(df.Target.nunique())

# Classificação pelo Gini
if gini >= 0.8:
    classe_gini = 'Totalmente desbalanceada'
elif gini >= 0.6:
    classe_gini = 'Altamente desbalanceada'
elif gini >= 0.4:
    classe_gini = 'Moderadamente desbalanceada'
else:
    classe_gini = 'Balanceada'

# Classificação pelo Shannon
if shannon <= 0.2 * max_shannon:
    classe_shannon = 'Totalmente desbalanceada'
elif shannon <= 0.4 * max_shannon:
    classe_shannon = 'Altamente desbalanceada'
elif shannon <= 0.7 * max_shannon:
    classe_shannon = 'Moderadamente desbalanceada'
else:
    classe_shannon = 'Balanceada'

# Ajuste final da classificação considerando Gini e Shannon
if classe_gini == 'Totalmente desbalanceada' or classe_shannon == 'Totalmente desbalanceada':
    classe_final = 'Totalmente desbalanceada'
elif classe_gini == 'Altamente desbalanceada' or classe_shannon == 'Altamente desbalanceada':
    classe_final = 'Altamente desbalanceada'
elif classe_gini == 'Moderadamente desbalanceada' or classe_shannon == 'Moderadamente desbalanceada':
    classe_final = 'Moderadamente desbalanceada'
else:
    classe_final = 'Balanceada'

print(classe_final)

Moderadamente desbalanceada


Dado que a nossa variável alvo é moderadamente desbalanceada apenas, vamos mantê-la em sua proporção atual e começaremos a transformar os dados para a modelagem.

In [29]:
df.Target.value_counts(normalize=True)

Target
0    0.669411
1    0.330589
Name: proportion, dtype: float64

Considerando o tamanho do nosso dataset, não iremos realizar amostragem e usaremos todos os dados disponíveis, de forma a evitar prejudicar o modelo.

In [30]:
# Dividindo variáveis pelo tipo
vars_continuas = vars[:4]
vars_binarias = ['Tuition fees up to date', 'Scholarship holder', 'Target']
vars_nominais = ['Course', 'Application mode']

display(df[vars_continuas].head(5))
display(df[vars_binarias].head(5))
display(df[vars_nominais].head(5))

,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade)
0,6,14.500000,6,12.428571
1,4,11.600000,0,0.000000
2,0,0.000000,0,0.000000
3,7,12.591250,7,12.820000
4,6,12.933333,6,12.933333


,Tuition fees up to date,Scholarship holder,Target
0,1,1,0
1,1,0,1
2,1,0,1
3,1,1,0
4,1,0,0


,Course,Application mode
0,9238,1
1,9238,17
2,9254,17
3,9500,1
4,9500,1


In [31]:
# Criando base de validação com 5% dos dados
df_valid = df.sample(frac=.05, random_state=412)

# Criando base de desenvolvimento com 95% dos dados
df_desen = df.drop(df_valid.index)

print(f'Dimensões da base de desenvolvimento: {df_desen.shape}\nDimensões da base de validação: {df_valid.shape}')

Dimensões da base de desenvolvimento: (72692, 9)
Dimensões da base de validação: (3826, 9)


In [32]:
# Aplicando One-hot encoding para as variáveis nominais
for var in vars_nominais:
    dummies = pd.get_dummies(df_desen[var], prefix=var).astype(np.float64).reset_index(drop=True)
    df_desen = pd.concat([df_desen.drop(columns=[var]).reset_index(drop=True), dummies], axis=1)

df_desen.head(5)

,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Tuition fees up to date,Scholarship holder,Target,Course_33,Course_39,Course_171,...,Application mode_18,Application mode_26,Application mode_27,Application mode_35,Application mode_39,Application mode_42,Application mode_43,Application mode_44,Application mode_51,Application mode_53
0,6,14.500000,6,12.428571,1,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,11.600000,0,0.000000,1,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.000000,0,0.000000,1,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,12.591250,7,12.820000,1,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,12.933333,6,12.933333,1,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Padronizando variáveis contínuas
scaler = StandardScaler()
df_desen[vars_continuas] = scaler.fit_transform(df_desen[vars_continuas])

df_desen.head(5)

,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Tuition fees up to date,Scholarship holder,Target,Course_33,Course_39,Course_171,...,Application mode_18,Application mode_26,Application mode_27,Application mode_35,Application mode_39,Application mode_42,Application mode_43,Application mode_44,Application mode_51,Application mode_53
0,0.676571,0.855142,0.717338,0.504739,1,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.067045,0.304253,-1.445177,-1.736160,1,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.554278,-1.899306,-1.445177,-1.736160,1,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.048379,0.492552,1.077757,0.575314,1,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.676571,0.557535,0.717338,0.595749,1,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# Separando base de desenvolvimento em features (X) e target (y)
X = df_desen.drop(columns=['Target'])
y = df_desen['Target']

display(X.head(5))
display(y.head(5))

,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Tuition fees up to date,Scholarship holder,Course_33,Course_39,Course_171,Course_979,...,Application mode_18,Application mode_26,Application mode_27,Application mode_35,Application mode_39,Application mode_42,Application mode_43,Application mode_44,Application mode_51,Application mode_53
0,0.676571,0.855142,0.717338,0.504739,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.067045,0.304253,-1.445177,-1.736160,1,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.554278,-1.899306,-1.445177,-1.736160,1,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.048379,0.492552,1.077757,0.575314,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.676571,0.557535,0.717338,0.595749,1,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


0    0
1    1
2    1
3    0
4    0
Name: Target, dtype: int64

In [39]:
# Criando objeto para reduzir a dimensão das variáveis
pca = PCA(n_components=20)

componentes_principais = pca.fit(X)

print(f'Variância Explicada por Componentes na Base:\n{componentes_principais.explained_variance_ratio_}')

Variância Explicada por Componentes na Base:
[0.62015529 0.05573732 0.04792439 0.03519079 0.02614666 0.02331551
 0.02034058 0.01790049 0.01563955 0.01389498 0.01307507 0.01264588
 0.01100134 0.01081228 0.00858178 0.00826044 0.00779104 0.00713562
 0.00702815 0.00616126]


In [40]:
componentes_principais.explained_variance_ratio_.sum()

0.9687384179851429

In [41]:
# Transformando base de desenvolvimento
X = pd.DataFrame(componentes_principais.transform(X), columns=[f'PC_{num+1}' for num in range(len(componentes_principais.components_))])
X.head(5)

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,PC_11,PC_12,PC_13,PC_14,PC_15,PC_16,PC_17,PC_18,PC_19,PC_20
0,1.478972,0.547006,-0.147478,-0.251189,-0.666344,0.362348,-0.277344,0.629891,-0.110391,-0.344433,-0.026940,-0.084809,-0.048030,0.027822,-0.043938,0.031299,0.135226,0.018731,0.048860,0.041933
1,-1.477547,-0.731375,-0.062578,-0.651192,0.211925,1.587719,0.356718,0.761619,-0.158361,-0.558020,-0.457485,-0.123767,0.006623,0.078062,-0.065733,-0.084325,-0.041881,-0.059504,-0.098466,0.002529
2,-3.303070,-0.308924,0.479842,-0.865275,0.308072,-0.031273,0.024997,0.010224,-0.160157,0.333798,-0.065366,0.694059,-0.431847,0.047211,-0.209119,-0.047662,0.153874,-0.032329,-0.013040,-0.035555
3,1.745914,0.815123,0.608678,-0.021032,-0.362030,-0.077437,0.552697,-0.049129,-0.042949,-0.020572,0.015824,-0.039563,-0.037046,-0.010451,0.006935,-0.064502,-0.104154,-0.043404,-0.031533,0.019088
4,1.344477,0.561156,0.115316,0.247557,0.438323,-0.211285,0.560974,0.009488,-0.208811,-0.193895,-0.027103,-0.019296,-0.068822,-0.046622,-0.024915,-0.048265,-0.088142,-0.039636,-0.056146,-0.034942


In [42]:
def criar_tabela_woe(variavel: pd.Series, resposta: pd.Series) -> tuple:
    '''
    Função para construir a tabela cruzada e calcular o Weight of Evidence das classes e Information Value da variável.

    Parâmetros:
    - variavel: uma Series do Pandas, sendo essa a variável a ser estudada.
    - resposta: uma Series do Pandas, sendo essa a variável resposta binária a ser usada como base.

    Retorna:
    - tupla contendo a tabela cruzada e o cálculo de information value.
    '''
    # Cria a tabela cruzada
    tab = pd.crosstab(variavel, resposta, margins=True)

    # Calculada as probabilidades do evento e não-evento
    tab['Prop_nao_evento'] = tab[0] / tab[0].sum()
    tab['Prop_evento'] = tab[1] / tab[1].sum()

    # Define um valor para impedir zeros estruturais no cálculo
    epsilon = 1e-10

    # Calcula Weight of Evidence e Information Value parcial das classes
    tab['WOE'] = np.log((tab['Prop_nao_evento'] + epsilon) / (tab['Prop_evento'] + epsilon))
    tab['IV_parcial'] = (tab['Prop_nao_evento'] - tab['Prop_evento']) * tab['WOE']

    # Calcula o Information Value da variável categórica
    iv = tab['IV_parcial'].sum()

    return iv, tab

def classificar_iv(iv: float) -> str:
    '''
    Função para classificar o IV de acordo com a classificação de Naeem Siddiqi.

    Parâmetros:
    - iv: o valor de Information Value da variável.

    Retorna:
    - string com a classificação de IV da variável.
    '''
    if iv < 0.02:
        return 'Inútil'
    elif 0.02 <= iv < 0.1:
        return 'Fraca'
    elif 0.1 <= iv < 0.3:
        return 'Média'
    elif 0.3 <= iv < 0.5:
        return 'Forte'
    return 'Sobreajuste'

In [45]:
# Calcular Information Value dos componentes principais
for comp in X:
    iv, _ = criar_tabela_woe(X[comp], y)
    print(f'IV do componente {comp} igual a : {iv:.2%} - {classificar_iv(iv)}')

IV do componente PC_1 igual a : 984.16% - Sobreajuste
IV do componente PC_2 igual a : 984.16% - Sobreajuste
IV do componente PC_3 igual a : 984.16% - Sobreajuste
IV do componente PC_4 igual a : 984.10% - Sobreajuste
IV do componente PC_5 igual a : 984.14% - Sobreajuste
IV do componente PC_6 igual a : 984.16% - Sobreajuste
IV do componente PC_7 igual a : 984.16% - Sobreajuste
IV do componente PC_8 igual a : 984.11% - Sobreajuste
IV do componente PC_9 igual a : 984.16% - Sobreajuste
IV do componente PC_10 igual a : 984.16% - Sobreajuste
IV do componente PC_11 igual a : 984.16% - Sobreajuste
IV do componente PC_12 igual a : 984.16% - Sobreajuste
IV do componente PC_13 igual a : 984.14% - Sobreajuste
IV do componente PC_14 igual a : 984.10% - Sobreajuste
IV do componente PC_15 igual a : 984.14% - Sobreajuste
IV do componente PC_16 igual a : 984.14% - Sobreajuste
IV do componente PC_17 igual a : 984.17% - Sobreajuste
IV do componente PC_18 igual a : 984.19% - Sobreajuste
IV do componente PC